In [11]:
import torch
from model import *
from dataloader import *
from utils.pyart import *
import argparse
import numpy as np
from pathlib import Path

data_path='./data/Multi_2dim_log_spiral'
checkpoint= './output/0119_3/checkpoint_50.pth'

# load checkpoint
checkpoint = torch.load(checkpoint)
branchNum = checkpoint['branchNum']
input_dim = checkpoint['input_dim']
branchLs = bnum2ls(branchNum)
n_joint = len(branchLs)

# load model
model = Model(branchLs, input_dim)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

Model(
  (q_layer): q_layer(
    (layers): ModuleList(
      (0): Linear(in_features=2, out_features=4, bias=True)
      (1): Linear(in_features=4, out_features=8, bias=True)
      (2): Linear(in_features=8, out_features=16, bias=True)
      (3): Linear(in_features=16, out_features=32, bias=True)
      (4): Linear(in_features=32, out_features=64, bias=True)
      (5): Linear(in_features=64, out_features=128, bias=True)
      (6): Linear(in_features=128, out_features=256, bias=True)
      (7): Linear(in_features=256, out_features=128, bias=True)
      (8): Linear(in_features=128, out_features=64, bias=True)
      (9): Linear(in_features=64, out_features=32, bias=True)
      (10): Linear(in_features=32, out_features=16, bias=True)
      (11): Linear(in_features=16, out_features=8, bias=True)
      (12): Linear(in_features=8, out_features=4, bias=True)
      (13): Linear(in_features=4, out_features=40, bias=True)
    )
  )
  (trans_layer): TransformLayer(
    (joint_1): PRjoint()
    (joi

In [14]:
# get input

test_data_loader = FoldToyDataloader(data_path, 8, -1, 1, 16)
input,_ = next(iter(test_data_loader))
input

tensor([[1.1395, 2.0966],
        [1.6068, 2.3164],
        [1.5202, 1.1919],
        [1.0523, 0.4356],
        [1.1640, 1.5206],
        [2.5112, 0.8905],
        [3.2059, 2.7037],
        [2.3566, 1.0878],
        [2.5322, 2.9130],
        [2.0712, 1.8193],
        [2.1500, 2.4039],
        [2.7673, 1.5165],
        [1.5724, 0.5782],
        [1.7506, 2.6466],
        [2.4031, 1.2477],
        [1.7861, 0.5366]])

In [17]:
# get postion of joint
TrackingSE3, RevSE3, PriSE3 = model(input)

In [73]:
def VecLoss(vec_tar,vec):
    vec_norm = torch.linalg.norm(vec,dim=1)
    vec = vec/ (vec_norm.unsqueeze(1))

    vec_tar_norm = torch.linalg.norm(vec_tar,dim=1)
    vec_tar = vec_tar/ (vec_tar_norm.unsqueeze(1))

    # check dot product
    res = vec * vec_tar
    res = torch.sum(res)
    
    return res

In [93]:
batch_size = TrackingSE3.size()[0]
device = TrackingSE3.device

Loss = torch.tensor(0).to(torch.float).to(device)
currJoint = 1
prev_tar = torch.tensor([[0,0,0]]).to(torch.float).to(device)
prev_pri_p = torch.tensor([[0,0,0]]).to(torch.float).to(device)
prev_rev_p = torch.tensor([[0,0,0]]).to(torch.float).to(device)

targetNums = TrackingSE3.size()[1]
for targetNum in range(targetNums):
    curr_tar = t2p(TrackingSE3[:,targetNum])
    vec_tar = curr_tar - prev_tar
    freeJnum = branchNum[targetNum]
    for joint in range(currJoint,currJoint+freeJnum+1):
        rev_p = t2p(RevSE3[:,joint-1])
        vec = (rev_p - prev_pri_p)
        Loss = Loss + VecLoss(vec_tar,vec)
        prev_rev_p = rev_p

        pri_p = t2p(PriSE3[:,joint-1])
        vec = (pri_p - prev_rev_p)
        Loss = Loss + VecLoss(vec_tar,vec)
        prev_pri_p = pri_p

        currJoint = currJoint + 1
     

Loss

tensor(212.6009, grad_fn=<AddBackward0>)

In [94]:
currJoint

21

In [64]:
# check if normalized to 1
vec_norm = torch.linalg.norm(vec,dim=1)
vec_temp = vec/ (vec_norm.unsqueeze(1))

vec_tar_norm = torch.linalg.norm(vec_tar,dim=1)
vec_tar_temp = vec_tar/ (vec_tar_norm.unsqueeze(1))

torch.linalg.norm(vec_temp,dim=1), torch.linalg.norm(vec_tar_temp,dim=1)


(tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        grad_fn=<CopyBackwards>),
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        grad_fn=<CopyBackwards>))

In [72]:
# check dot product
res = vec_temp * vec_tar_temp
res = torch.sum(res)

In [32]:
RevSE3[:,joint]

tensor([[[-0.1814, -0.6987, -0.6921, -0.9080],
         [ 0.3767, -0.6994,  0.6073,  0.1508],
         [-0.9084, -0.1506,  0.3901,  0.9647],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[-0.1819, -0.6997, -0.6908, -0.3908],
         [ 0.3767, -0.6985,  0.6084,  0.3884],
         [-0.9083, -0.1496,  0.3907,  1.0845],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[-0.1886, -0.6998, -0.6890, -0.4879],
         [ 0.3746, -0.6998,  0.6082,  0.3409],
         [-0.9078, -0.1434,  0.3941,  1.0618],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[-0.2064, -0.6841, -0.6996, -1.4439],
         [ 0.3661, -0.7170,  0.5932, -0.1555],
         [-0.9074, -0.1337,  0.3984,  0.8364],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[-0.1845, -0.6989, -0.6910, -0.8643],
         [ 0.3758, -0.6998,  0.6075,  0.1688],
         [-0.9082, -0.1476,  0.3918,  0.9747],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[-0.1959, -0.7016, -0.6851,  0.4428],
   